In [ ]:
# === colab configuration ===
# p.s. when training on colab, weights are saved on Drive (directory DSCAS/weights).
# p.p.s. skip this cell if running demo file locally!

! pip install torch-summary
! pip install ray
! pip install tensorboardX==2.5.1
! pip install tensorboard==2.11.2
from google.colab import drive, files
import sys

# setting paths
repository_path = '/content/deep-seasonal-color-analysis-system/'
dataset_path = repository_path + 'headsegmentation_dataset_ccncsa/'
dataset_path_drive = '/content/drive/MyDrive/DSCAS/headsegmentation_dataset_ccncsa/'
weights_path = repository_path + 'models/weights/'
weights_path_drive = '/content/drive/MyDrive/DSCAS/weights/'
checkpoint_path = repository_path + 'models/loss_selection/'
checkpoint_archive = '/content/' + checkpoint_path.split('/')[-2] + '.zip'

sys.path.insert(0, repository_path)

# cloning project repository and downloading dataset
drive.mount('/content/drive')
! test ! -d $repository_path && git clone https://github.com/mrcmich/deep-seasonal-color-analysis-system.git
! test ! -d $dataset_path && cp -R $dataset_path_drive $dataset_path
%cd $repository_path

# setting branch and pulling updates
branch = 'main__fastscnn_hyperparameters_tuning'
! git checkout $branch
! git pull origin $branch

executing_on_colab = True

In [ ]:
import os
import torch
from torch import nn, optim
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from models import dataset, training_and_testing
from models.local.FastSCNN.models import fast_scnn
from metrics_and_losses import metrics
from utils import segmentation_labels, custom_transforms, model_names
from models import config
from slurm_scripts import slurm_config
from ray import tune
from ray.tune import CLIReporter
from functools import partial

try:
  executing_on_colab
except NameError:
  executing_on_colab = False
  weights_path = config.WEIGHTS_PATH
  dataset_path = config.DATASET_PATH
  checkpoint_path = config.LOSS_SELECTION_PATH

In [ ]:
# defining transforms
image_transform = T.Compose([T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])
target_transform = T.Compose([T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO)])

# fetching dataset
n_classes = len(segmentation_labels.labels)
img_paths, label_paths = dataset.get_paths(dataset_path, file_name=config.DATASET_INDEX_NAME)
X_train, _, Y_train, _ = train_test_split(
    img_paths, label_paths, test_size=0.20, random_state=99, shuffle=True)
train_dataset = dataset.CcncsaDataset(X_train, Y_train, image_transform, target_transform)

# model parameters
model_name = "fastscnn"
optimizer = optim.Adam

# setting up hyperparameters for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_epochs = 10
batch_size = 32
score_fn = metrics.batch_mIoU
learning_rate = 0.01
class_weights = torch.tensor(config.CLASS_WEIGHTS, device=device)

# Ray Tune parameters
cpus_per_trial = 0
gpus_per_trial = torch.cuda.device_count()
num_samples = 1  # Number of times each combination is sampled (n_epochs are done per sample)
metric = "val_loss"
metrics_columns = ["train_loss", "train_score", "val_loss", "val_score", "training_iteration"]
local_dir = checkpoint_path + model_names.MODEL_NAMES[model_name]
max_report_frequency = 600
reporter = CLIReporter(
    metric_columns=metrics_columns, max_report_frequency=max_report_frequency)

In [ ]:
# === training with std. loss ===

checkpoint_dir = local_dir if executing_on_colab else (os.path.abspath('./' + local_dir) + '/')
cfg = {
    "lr": learning_rate,
    "lr_scheduler": "none",
    "batch_size": batch_size,
    "loss": "std",
    "from_checkpoint": False,
    "checkpoint_dir": checkpoint_dir
    }

model = fast_scnn.FastSCNN(n_classes)
loss_fn = nn.CrossEntropyLoss()

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab:
    files.download(checkpoint_archive)

In [ ]:
# === training with std. loss ===

cfg["loss"] = "weighted"

model = fast_scnn.FastSCNN(n_classes)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab:
    files.download(checkpoint_archive)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {checkpoint_path}